In [17]:
from mtcnn import MTCNN
import cv2
import os
from keras_facenet import FaceNet
import tensorflow as tf
import numpy as np
from PIL import Image
from numpy import asarray
import pickle

In [2]:
#proses image yang belom dicropping mukanya
def make_database_should_crop(path):
    database = {}
    detector = MTCNN()
    for image in os.listdir(path):
        path_image = os.path.join(path, folder, image)
        img = cv2.imread(path_image)
        posisi = detector.detect_faces(img
        x1, y1, w, h = posisi[0]['box']
        x2, y2 = x1 + w, y1 + h
        img = img[y1:y2, x1:x2]
        img = Image.fromarray(img)
        img = img.resize((160, 160))
        img = asarray(img)
        img = np.expand_dims(img, axis=0)
        database[image.split('.')[0]] = my_facenet.embeddings(img)[0]
    return database

In [3]:
#proses image yang udah dicropping mukanya
my_facenet = FaceNet()
def make_database(path):
    database = {}
    for image in os.listdir(path):
        path_image = os.path.join(path, image)
        img = cv2.imread(path_image)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = img.resize((160, 160))
        img = asarray(img)
        img = np.expand_dims(img, axis=0)
        database[image.split('.')[0]] = my_facenet.embeddings(img)[0]
    return database

In [18]:
def simpan_database(database):
    myfile = open("data.pkl", "wb")
    pickle.dump(database, myfile)
    myfile.close()

In [19]:
database = make_database('Data')

1/1 [==============================] - 0s 175ms/step


In [20]:
simpan_database(database)

In [8]:
def cos_similarity(anchor, test):
    a = np.matmul(np.transpose(anchor), test)
    b = np.sum(np.multiply(anchor, test))
    c = np.sum(np.multiply(test, test))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [9]:
img = cv2.imread('Sample8_60.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = Image.fromarray(img)
img = img.resize((160, 160))
img = asarray(img)
img = np.expand_dims(img, axis=0)
test_embedder = my_facenet.embeddings(img)[0]

distance = {}
for key in database:
    #jarak = euclidean_distance(database.get(key), test_embedder)
    jarak = findCosineSimilarity(database.get(key), test_embedder)
    print(jarak)
    distance[key] = jarak

1/1 [==============================] - 0s 166ms/step
0.42614173889160156
0.7522075474262238
0.5600938200950623


In [10]:
distance

{'Dirga': 0.42614173889160156,
 'Jen': 0.7522075474262238,
 'Ronald': 0.5600938200950623}

In [11]:
detector = cv2.CascadeClassifier('haarcascade.xml')

In [16]:
vid = cv2.VideoCapture(0)
while(True):
    ret, frame = vid.read()
    wajah = detector.detectMultiScale(frame, scaleFactor = 1.2, minNeighbors = 5)
    for x1, y1, w, h in wajah:
        x2 = x1 + w
        y2 = y1 + h
        muka = frame[y1:y2, x1:x2]
        muka = cv2.cvtColor(muka, cv2.COLOR_BGR2RGB)
        muka = Image.fromarray(muka)
        muka = muka.resize((160,160)) 
        muka = asarray(muka)
        muka = np.expand_dims(muka, axis=0)
        test_embedder = my_facenet.embeddings(muka)[0]
        i = 100
        k = 'unknown'
        for key in database:
            distance = findCosineSimilarity(test_embedder, database.get(key))
            if (distance < i) & (distance < 0.5):
                i = distance
                k = key
        frame = cv2.rectangle(frame, (x1,y1), (x2,y2), (255,255,255), 1)
        frame = cv2.putText(frame, k, (x1,y1), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        cv2.imshow('frame', frame)
    
        if cv2.waitKey(2) & 0xFF == ord('q'):
            break
  
vid.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 165ms/step


KeyboardInterrupt: 